In [112]:
import pyforest
from my_module import *

pd.set_option('display.max_rows', None)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error
from lazypredict.Supervised import LazyRegressor
from pandas.plotting import scatter_matrix
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [159]:
df = pd.read_csv("billets.csv", sep=";")
df.head()

,is_genuine,diagonal,height_left,height_right,margin_low,margin_up,length
0,True,171.81,104.86,104.95,4.52,2.89,112.83
1,True,171.46,103.36,103.66,3.77,2.99,113.09
2,True,172.69,104.48,103.50,4.40,2.94,113.16
3,True,171.36,103.91,103.94,3.62,3.01,113.51
4,True,171.73,104.28,103.46,4.04,3.48,112.54


In [162]:
X = df.drop("is_genuine", axis=1)
y = df["is_genuine"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=3, test_size=0.20
)

In [194]:
rf = Pipeline(
    steps=[
        ("imputer", IterativeImputer()),
        (
            "classifier",
            RandomForestClassifier(
                criterion="gini", max_depth=7, max_features="auto", n_estimators=200
            ),
        ),
    ]
)

<IPython.core.display.Javascript object>

In [200]:
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
print(f"Accuracy {name}: " + str(metrics.accuracy_score(y_test, preds)))
print(f"F1 score {name}: " + str(metrics.f1_score(y_test, preds, average="macro")))

Accuracy LogisticRegression: 0.9933333333333333
F1 score LogisticRegression: 0.992422711658921


In [201]:
df_verif = pd.read_csv("billets_production.csv")
id = df_verif["id"]
df_verif.drop("id", axis=1, inplace=True)
df_verif.head()

,diagonal,height_left,height_right,margin_low,margin_up,length
0,171.76,104.01,103.54,5.21,3.30,111.42
1,171.87,104.17,104.13,6.00,3.31,112.09
2,172.00,104.58,104.29,4.99,3.39,111.57
3,172.49,104.55,104.34,4.44,3.03,113.20
4,171.65,103.63,103.56,3.77,3.16,113.33


In [203]:
# Vérification RandomForest
df11 = pd.DataFrame(rf.predict(df_verif))
df11.columns = ["is_genuine"]
df11["is_genuine"] = np.where(df11["is_genuine"] == 1 , True, False)
df12 = pd.DataFrame(rf.predict_proba(df_verif))
df12.columns = ["proba_false", "proba_true"]
df12["id"] = id
df11.join(df12)

,is_genuine,proba_false,proba_true,id
0,False,1.00,0.00,A_1
1,False,1.00,0.00,A_2
2,False,0.99,0.01,A_3
3,True,0.03,0.97,A_4
4,True,0.00,1.00,A_5
